# BITCOIN BOT PROJECT - BACKTEST NOTEBOOK 2
The goal of this project is to develop a working trading bot with the binance api. The signaller is supertrend on the 1m timescale.

## Project Plan
#### PHASE 1 - DATA COLLECTION AND BACKTESTS
1. Data Collection
2. Data Clean
3. Build Supertrend Signal
4. Carry out back test
5. Tune back test hyperparameters
6. Investigate optimisation
7. Retune hyperparameters
8. Evaluate the strategy - Bot / no bot

#### PHASE 2 - BUILD AND TEST BOT IN LOCAL ENV
1. Review binance bot options and develop bot type / operation and functionality
2. Build bot
3. Test strategy offline
4. Debug and optimise
5. Add new data to initial phase 1 backtest to see if results are comparable and hyperparameters similar or unchanged
6. Debug / optimise / retest in local env


#### PHASE 3 - LIVE TESTS
1. Commence live tests with small initial capital and low risk
2. Add new data to initial phase 1 backtest to see if results are comparable and hyperparameters similar or unchanged
3. Debug / optimise / retest in live environment

#### PHASE 4 - RUN BOT
1. RUN BOT WITH FULLY OPTIMISED STRATEGY / HYPERPARAMETERS / RISK 
2. COLLECT GAINS

## Notebook Aim
Develop / fine tune the back test strategy initiated in notebook #1.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import time
import warnings
import math

from tqdm import tqdm
from collections import Counter

pd.set_option("display.max_columns", None)

warnings.filterwarnings("ignore")

In [3]:
# Read in the supertrend dataframe
df = pd.read_csv("supertrend.csv", infer_datetime_format=['time'], parse_dates=['time'])

In [10]:
print(df.shape)
df.loc[8:18]

(23992, 12)


,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct
8,2022-03-14 00:08:00,37720.99,37728.19,37707.02,37714.94,NaN,NaN,no_signal,nosig,NaN,NaN,NaN
9,2022-03-14 00:09:00,37723.24,37725.89,37699.00,37712.00,NaN,37839.792000,sell,sellsig,0.00,0.00,0.000
10,2022-03-14 00:10:00,37712.00,37727.13,37703.99,37721.57,NaN,37837.114300,sell,nosig,-9.57,8.01,0.021
11,2022-03-14 00:11:00,37721.57,37725.00,37700.00,37719.75,NaN,37829.398870,sell,nosig,-7.75,12.00,0.032
12,2022-03-14 00:12:00,37719.59,37724.30,37680.11,37680.11,NaN,37820.670983,sell,nosig,31.89,31.89,0.085
13,2022-03-14 00:13:00,37683.86,37688.56,37560.40,37630.91,NaN,37769.547385,sell,nosig,81.09,151.60,0.402
14,2022-03-14 00:14:00,37633.93,37644.22,37590.82,37633.30,NaN,37764.100646,sell,nosig,78.70,121.18,0.321
15,2022-03-14 00:15:00,37636.32,37755.30,37627.89,37750.62,NaN,37764.100646,sell,nosig,-38.62,84.11,0.223
16,2022-03-14 00:16:00,37750.61,37846.00,37750.61,37839.84,37616.556977,NaN,buy,buysig,0.00,0.00,0.000
17,2022-03-14 00:17:00,37837.13,37895.69,37817.26,37852.01,37669.372779,NaN,buy,nosig,12.17,55.85,0.148


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23992 entries, 0 to 23991
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       23992 non-null  datetime64[ns]
 1   open       23992 non-null  float64       
 2   high       23992 non-null  float64       
 3   low        23992 non-null  float64       
 4   close      23992 non-null  float64       
 5   uptrend    12727 non-null  float64       
 6   downtrend  11257 non-null  float64       
 7   signal     23992 non-null  object        
 8   trigger    23991 non-null  object        
 9   close_pl   23983 non-null  float64       
 10  max_pl     23983 non-null  float64       
 11  max_pct    23983 non-null  float64       
dtypes: datetime64[ns](1), float64(9), object(2)
memory usage: 2.2+ MB


#### PLAN
Initially we want to simplify some of the code and review the performance evaluation metrics. 

In [14]:
# Get the signal trigger set to get the trade start/finish indices
trigs = df[(df.trigger=="sellsig") | (df.trigger=="buysig")]

In [15]:
trigs.head()

,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct
9,2022-03-14 00:09:00,37723.24,37725.89,37699.00,37712.00,NaN,37839.792000,sell,sellsig,0.0,0.0,0.0
16,2022-03-14 00:16:00,37750.61,37846.00,37750.61,37839.84,37616.556977,NaN,buy,buysig,0.0,0.0,0.0
84,2022-03-14 01:24:00,38080.27,38094.26,38037.45,38059.44,NaN,38218.679324,sell,sellsig,0.0,0.0,0.0
116,2022-03-14 01:56:00,38075.68,38087.78,38069.85,38087.78,37974.284582,NaN,buy,buysig,0.0,0.0,0.0
122,2022-03-14 02:02:00,38047.23,38066.65,37990.01,37990.01,NaN,38152.364011,sell,sellsig,0.0,0.0,0.0


In [166]:
tp1=0.28
tp2=0.5
tp3=0.85
p1=25
p2=25
p3=50
        
# Create a list to store each trade result and some other metrics :))
results = []
outcome = []
max_pct_trade = []
tp_list = []

# List of trade trigger indexes
trade_idx = trigs.index


# Now we want to loop through the trade indexes and then back test the trades within the index windows
for i,j in enumerate(trade_idx[:-1]):
    
    tp1_hit=False
    tp2_hit=False
    tp3_hit=False
    
    
    # Set our capital to £100
    initial_capital = 100
    # Set our running capital to £100
    running_capital = 100

    # Set our profit slices give the strategy parameters
    profit_slices = []


    # Query the trade from our supertrend dataset
    trade = df.loc[j:trade_idx[i+1]]
    entry_price = trade.iloc[0].close
    
    # Set a max pct gain tracker for each trade - to be stored in max_pct list
    max_pct_gain = 0

    print("=======================================")
    # Loop through all rows in the isolated trade data
    for index, row in trade.iterrows():
        if row["max_pct"]>max_pct_gain:
            max_pct_gain=row["max_pct"]
        
        
#         print("===========================")
#         print("Trade index: i{} s{}, f{}, Signal direction {}".format(index, 
#                                                                       trade.index[0], 
#                                                                       trade.index[-1], 
#                                                                       trade.signal.iloc[0]))
        
    
        # Now we'll calc the tp winners

        # We need to start with an if statement per tp... elifs will break the strategy as a 
        # single candle could pass thro' tp1 to tp2 and we would not bank relevant profits at tp1!

        if row["max_pct"]>=tp1 and tp1_hit==False:
            
            # Assuming tp1,tp2,tp3 = 0.38,0.54,0.6 (tp1*10 =3.8%) and initial capital ==100 and p1,p2,p3 ==1,1,98
#             print(row["max_pct"], tp1)
            tp1_hit=True
            position = running_capital*(1+(tp1/10)) # 103.8
            profit_take = position*(p1/100) # 1.038
#             print("profit take {}".format(profit_take))
            profit_slices.append(profit_take) # Add 1.038
            # new_position = position*((100-p1)/100)
            new_position = position-profit_take # 103.8-1.038 = 102.76
#             print("new position {}".format(new_position))
            running_capital = new_position

        if row["max_pct"]>=tp2 and tp2_hit==False:
#             print(row["max_pct"], tp2)
            tp2_hit=True
            # Now we have moved from tp1 to tp2 but we can only book the difference against our position size
            price_gain = tp2-tp1 # 0.54-0.38
            position = running_capital*(1+(price_gain/10)) # 102.76*1.016 = 104.4...
            profit_take = position*(p2/100) # 104.4*(1/100) = 1.044
            profit_slices.append(profit_take) # 1.044
            # new_position = position*((1-p2)/10)
            new_position = position-profit_take # 103.362
            running_capital = new_position

        if row["max_pct"]>=tp3 and tp3_hit==False:
            tp3_hit=True
            # Now we have moved from tp2 to tp3 but we can only book the difference against our position size
            price_gain = tp3-tp2 # 0.6-0.54=0.06
            position = running_capital*(1+(price_gain/10)) # 103.362*1.006 = 103.98
            # profit_take = position*(p3/100)
            # Now we close the trade at tp3 so we take whatever profit is on the table and add to profit slice
            profit_take = position-100 # ~3.98
            # Add the profit
            profit_slices.append(profit_take)
            # new_position = position*((1-p3)/10)
            # WE NO LONGER HAVE A POSITION OPEN AT THIS POINT AS ALL RISK HAS BEEN REMOVED FROM TABLE
            #new_position = position-profit_take
            #capital = new_position
        
        
        # First we calculate the stops and losses

        # If we are on the last candle of the trade and we haven't hit any tps - we have a loser
        # The nature of the data is that the supertrend reversal / stop is hit on the last candle

        if index == trade.index[-1] and tp1_hit==False:
            ## SIMPLY IF WE ARE ON THE LAST CANDLE HAVE HIT NO TPS WE WILL STOP OUT AT THE SUPERTREND
            ## The %age drop/gain can be calculated by the entry price and the close price of last candle
            
            # Calculate pct loss for a buy trade stop out
            if trade.iloc[0].signal=="buy":
                factor = row['close']/entry_price
                pct_loss = (factor-1)*100
                profit_slices.append(pct_loss*10)
            
            # Calculate pct loss for a sell trade stop out
            elif trade.iloc[0].signal=="sell":
                factor = row['close']/entry_price
                pct_loss = (factor-1)*(-100)
                profit_slices.append(pct_loss*10)
                
            
        
        
        elif index == trade.index[-1] and tp2_hit == True and tp3_hit == False:
            
            if trade.iloc[0].signal=="buy":
                
                # Calculate buys closing at stop
                tp2_price = entry_price*(1+(tp2/10))
                close_price = row["close"]
                
                factor = close_price/tp2_price
                
                final_position = running_capital*factor # 103.362*0.9298 = 96.105
                
                pct_loss = ((final_position/initial_capital)-1)*100 # ((96.105/100)-1)*100 = -3.89
                
                #pct_loss = (factor-1)*100
                profit_slices.append(pct_loss)
                
            
            elif trade.iloc[0].signal=="sell": 
                
                # Calculate sells closing at stop
                tp2_price = entry_price*(1-(tp2/10))
                
                close_price = row["close"]
                
                # The following is convoluted because we're trying to calculate a pct drop of position
                # on an actual price rise
                factor = 1-(abs(1-(close_price/tp2_price)))
                final_pos = running_capital*factor
                pct_loss = final_pos-100
                
                profit_slices.append(pct_loss)
        
            
                    
        
        elif index == trade.index[-1] and tp1_hit == True and tp2_hit==False and tp3_hit==False:
            # BUYS ... remember buys have a sell signal on there last row signalling reversal
            if trade.iloc[0].signal=="buy":
                
                # Calculate buys closing at stop
                tp1_price = entry_price*(1+(tp1/10))
                close_price = row["close"]
                
                factor = close_price/tp1_price
                
                final_position = running_capital*factor # 103.362*0.9298 = 96.105
                
                pct_loss = ((final_position/initial_capital)-1)*100 # ((96.105/100)-1)*100 = -3.89
                
                #pct_loss = (factor-1)*100
                profit_slices.append(pct_loss)
                
                
            elif trade.iloc[0].signal=="sell":
                
                # Calculate sells closing at stop
                tp1_price = entry_price*(1-(tp1/10))
                
                close_price = row["close"]
                
                # The following is convoluted because we're trying to calculate a pct drop of position
                # on an actual price rise
                factor = 1-(abs(1-(close_price/tp1_price)))
                final_pos = running_capital*factor
                pct_loss = final_pos-100
                
                profit_slices.append(pct_loss)
                
            

        
            
#         print(tp1_hit, tp2_hit, tp3_hit)
    print(trade.iloc[0].time)        
    print(tp1_hit, tp2_hit, tp3_hit)
    print(profit_slices)
    trade_profit = sum(profit_slices)
    if trade_profit>100:
        outcome.append("win")
    else:
        outcome.append("loss")
    max_pct_trade.append(max_pct_gain)
    tp_list.append([tp1_hit, tp2_hit, tp3_hit])
    # Append the sum of our profit list to the trade results list
    results.append(trade_profit)







2022-03-14 00:09:00
True False False
[25.7, -25.38987806219457]
2022-03-14 00:16:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-14 01:24:00
True False False
[25.7, -25.18005202284884]
2022-03-14 01:56:00
False False False
[-2.566965047582137]
2022-03-14 02:02:00
False False False
[-4.7767821066644345]
2022-03-14 02:35:00
False False False
[-2.6844649460802117]
2022-03-14 02:47:00
False False False
[-0.26767178868047914]
2022-03-14 03:30:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-14 04:31:00
False False False
[-4.690773389310454]
2022-03-14 05:11:00
False False False
[-2.858583419438898]
2022-03-14 05:35:00
False False False
[-1.9908688185823475]
2022-03-14 06:05:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-14 08:15:00
False False False
[-1.96202361248754]
2022-03-14 08:46:00
False False False
[-2.8942804565439406]
2022-03-14 09:07:00
False False False
[-2.6213961257028906]
2022-03-14 09:20:00
True True False
[25.7, 19.69905, -43.

2022-03-16 18:39:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-16 21:15:00
True True False
[25.7, 19.69905, -43.976606858092694]
2022-03-16 21:51:00
False False False
[-3.5008278214002653]
2022-03-16 22:17:00
True True False
[25.7, 19.69905, -43.86016294522672]
2022-03-16 23:03:00
True True False
[25.7, 19.69905, -43.624397951769645]
2022-03-17 00:07:00
True False False
[25.7, -25.13233672126968]
2022-03-17 00:59:00
False False False
[-2.82921936064362]
2022-03-17 01:04:00
False False False
[-1.8046540228460195]
2022-03-17 01:33:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-17 01:50:00
False False False
[-7.7077365361706285]
2022-03-17 02:02:00
False False False
[-6.453667144036057]
2022-03-17 02:12:00
True False False
[25.7, -25.102201988776088]
2022-03-17 02:47:00
False False False
[-2.6385910911007615]
2022-03-17 03:10:00
False False False
[-2.885523975112214]
2022-03-17 03:24:00
False False False
[-2.005763683155082]
2022-03-17 03:45:00
True Fals

2022-03-18 14:37:00
False False False
[-3.2631538726068943]
2022-03-18 15:07:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-18 15:49:00
False False False
[-8.928416075511203]
2022-03-18 16:17:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-18 17:43:00
False False False
[-3.399566318811109]
2022-03-18 17:54:00
True True False
[25.7, 19.69905, -43.6466690236466]
2022-03-18 18:33:00
False False False
[-3.812081736259021]
2022-03-18 18:40:00
False False False
[-2.5442589310473718]
2022-03-18 19:12:00
False False False
[-4.18943621089185]
2022-03-18 19:31:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-18 20:10:00
True True False
[25.7, 19.69905, -43.841810093478074]
2022-03-18 21:11:00
False False False
[-3.266083737066494]
2022-03-18 21:18:00
False False False
[-1.071178634310721]
2022-03-18 21:40:00
False False False
[-0.45472190404982626]
2022-03-18 22:04:00
False False False
[-0.40632322258127296]
2022-03-18 22:53:00
True False False
[25

2022-03-20 13:11:00
True False False
[25.7, -25.125886600256578]
2022-03-20 13:45:00
False False False
[-2.6699783273022026]
2022-03-20 13:56:00
False False False
[-1.9728475649980481]
2022-03-20 14:03:00
False False False
[-1.492833772670732]
2022-03-20 14:39:00
True False False
[25.7, -25.086679053833663]
2022-03-20 15:11:00
False False False
[-1.0278205434005594]
2022-03-20 15:23:00
True False False
[25.7, -25.008613844674173]
2022-03-20 15:54:00
False False False
[-2.6845297182149785]
2022-03-20 16:06:00
False False False
[-2.447645167735324]
2022-03-20 16:12:00
True False False
[25.7, -25.04286871808401]
2022-03-20 16:21:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-20 18:04:00
False False False
[-1.1293038308375358]
2022-03-20 18:47:00
False False False
[-2.5211850080339993]
2022-03-20 18:51:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-20 20:38:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-20 21:16:00
False False False
[-1.95

2022-03-22 23:40:00
False False False
[-3.4310581941519036]
2022-03-23 00:06:00
False False False
[-1.6229428074672203]
2022-03-23 00:26:00
False False False
[-1.8500514204773388]
2022-03-23 00:48:00
False False False
[-3.006495264316289]
2022-03-23 01:00:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-23 01:41:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-23 02:45:00
True True False
[25.7, 19.69905, -43.6780095116719]
2022-03-23 04:06:00
False False False
[-3.4599361902453163]
2022-03-23 04:16:00
True True False
[25.7, 19.69905, -43.9067297567174]
2022-03-23 04:53:00
False False False
[-3.349268163467145]
2022-03-23 05:05:00
False False False
[-1.9474783167361398]
2022-03-23 05:18:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-23 07:06:00
True False False
[25.7, -25.108030764682823]
2022-03-23 07:26:00
False False False
[-2.1459171495770146]
2022-03-23 07:36:00
False False False
[-2.0794812869260326]
2022-03-23 07:39:00
False False Fa

2022-03-25 12:23:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-25 14:13:00
True False False
[25.7, -25.280991837912012]
2022-03-25 14:23:00
False False False
[-3.095677297477506]
2022-03-25 14:56:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-25 16:10:00
False False False
[-2.92504447123898]
2022-03-25 16:23:00
True True False
[25.7, 19.69905, -43.86878786814755]
2022-03-25 17:18:00
True True False
[25.7, 19.69905, -43.62561078887502]
2022-03-25 18:20:00
False False False
[-3.5063572320506164]
2022-03-25 18:44:00
False False False
[-1.9795811164100563]
2022-03-25 19:01:00
True False False
[25.7, -25.078737807627306]
2022-03-25 19:42:00
True False False
[25.7, -24.93310797884243]
2022-03-25 20:32:00
False False False
[-2.4571634283845345]
2022-03-25 20:42:00
True False False
[25.7, -25.055113606120983]
2022-03-25 21:09:00
False False False
[-0.32296859608016426]
2022-03-25 21:50:00
False False False
[-0.3545441187241538]
2022-03-25 22:00:00
True False 

2022-03-28 05:57:00
True True False
[25.7, 19.69905, -43.83657914088471]
2022-03-28 06:38:00
False False False
[-2.1880403866034737]
2022-03-28 06:51:00
False False False
[-1.9403271556228674]
2022-03-28 07:03:00
True False False
[25.7, -25.071791218301918]
2022-03-28 08:06:00
False False False
[-0.9370403884008116]
2022-03-28 08:27:00
True True False
[25.7, 19.69905, -43.55419750318175]
2022-03-28 10:35:00
False False False
[-2.4429649401633835]
2022-03-28 10:54:00
True False False
[25.7, -25.106219608816293]
2022-03-28 11:03:00
False False False
[-2.9813568194958506]
2022-03-28 11:27:00
False False False
[-0.6475180588410012]
2022-03-28 11:42:00
False False False
[-1.0519731424254442]
2022-03-28 12:02:00
False False False
[-1.4969528430270884]
2022-03-28 12:18:00
False False False
[-0.19279623533074997]
2022-03-28 13:07:00
True True True
[25.7, 19.69905, -38.834449750000005]
2022-03-28 14:06:00
True False False
[25.7, -25.203354462087347]
2022-03-28 14:29:00
True False False
[25.7, -

In [122]:
returns = sum(results)
break_even = 100*len(results)
gain = returns-break_even
pct_gain = returns/break_even
gain, pct_gain

num_wins = len([x for x in results if x>0])
num_losers = len([x for x in results if x<0])

print("This strategy has returned {}% over {} trades".format(sum(results), len(results)))
print("Winners: {}, Losers: {}".format(num_wins, num_losers))
print("Winrate: {}".format(num_wins/len(results)))
# print("At 10x leverage this is {}%!!".format(round(pct_gain*100-100,2)*10))

This strategy has returned -101.81945489246337% over 661 trades
Winners: 284, Losers: 377
Winrate: 0.4296520423600605


In [121]:
sum(results)

-101.81945489246337

In [123]:
results

[0.3101219378054303,
 6.564600249999998,
 0.5199479771511584,
 -2.566965047582137,
 -4.7767821066644345,
 -2.6844649460802117,
 -0.26767178868047914,
 6.564600249999998,
 -4.690773389310454,
 -2.858583419438898,
 -1.9908688185823475,
 6.564600249999998,
 -1.96202361248754,
 -2.8942804565439406,
 -2.6213961257028906,
 1.7841820492365414,
 0.6868152772612284,
 -1.3179370179108307,
 -1.9565846747435778,
 -2.6523530460124123,
 1.6705226357441347,
 0.6161653543732868,
 -3.1534410797713353,
 -6.124353655398984,
 -6.104937898045559,
 1.6996135172705849,
 0.3849865363996905,
 0.549944053452343,
 0.4882460430621727,
 -2.4409298316055983,
 1.5375114675210284,
 -2.9981694897774913,
 1.4327479207355864,
 1.9420863233452863,
 1.4379516384118602,
 -2.591811700355273,
 -2.1583472074993004,
 6.564600249999998,
 -9.51335082260596,
 -3.827244927559592,
 6.564600249999998,
 1.7186200695690914,
 1.5825992936386726,
 -1.6845011725828973,
 -0.9780916248163241,
 -2.1135622132192333,
 -2.0761946384688645,
 0.

In [136]:
res = pd.DataFrame(results)
res

,0
0,0.310122
1,6.564600
2,0.519948
3,-2.566965
4,-4.776782
...,...
656,0.739506
657,-0.662768
658,1.493509
659,-0.116029


In [101]:
idx1 = 15
idx2 = 16

df.loc[trigs.index[idx1]-1:trigs.index[idx2]]

,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct
559,2022-03-14 09:19:00,39011.00,39023.89,38998.95,38998.95,NaN,39047.088495,sell,nosig,-34.98,-34.98,-0.090
560,2022-03-14 09:20:00,38997.34,39068.67,38994.06,39066.11,38906.881058,NaN,buy,buysig,0.00,0.00,0.000
561,2022-03-14 09:21:00,39060.16,39061.44,39030.57,39034.18,38923.307452,NaN,buy,nosig,-31.93,-4.67,-0.012
562,2022-03-14 09:22:00,39034.17,39046.11,39033.16,39037.17,38925.322207,NaN,buy,nosig,-28.94,-20.00,-0.051
563,2022-03-14 09:23:00,39036.03,39047.95,39031.80,39031.80,38932.148486,NaN,buy,nosig,-34.31,-18.16,-0.046
564,2022-03-14 09:24:00,39031.81,39046.15,38996.73,39010.08,38932.148486,NaN,buy,nosig,-56.03,-19.96,-0.051
565,2022-03-14 09:25:00,39006.88,39050.00,39005.01,39050.00,38932.148486,NaN,buy,nosig,-16.11,-16.11,-0.041
566,2022-03-14 09:26:00,39050.00,39064.14,39035.10,39042.21,38938.219012,NaN,buy,nosig,-23.90,-1.97,-0.005
567,2022-03-14 09:27:00,39046.13,39047.28,39010.77,39012.20,38938.219012,NaN,buy,nosig,-53.91,-18.83,-0.048
568,2022-03-14 09:28:00,39017.30,39027.60,38998.27,39001.85,38938.219012,NaN,buy,nosig,-64.26,-38.51,-0.099


In [112]:
tp1=0.3
tp2=0.6
tp3=1
p1=40
p2=40
p3=20
        
# Create a list to store each trade result and some other metrics :))
results = []
outcome = []
max_pct_trade = []

# List of trade trigger indexes
trade_idx = trigs.index


# Now we want to loop through the trade indexes and then back test the trades within the index windows

tp1_hit=False
tp2_hit=False
tp3_hit=False

# Set our capital to £100
initial_capital = 100
# Set our running capital to £100
running_capital = 100

# Set our profit slices give the strategy parameters
profit_slices = []


# Query the trade from our supertrend dataset
trade = df.loc[559:602]
entry_price = trade.iloc[0].close

# Set a max pct gain tracker for each trade - to be stored in max_pct list
max_pct_gain = 0

print("=======================================")
# Loop through all rows in the isolated trade data
for index, row in trade.iterrows():
    print(tp1_hit, tp2_hit, tp3_hit)
    if index == trade.index[-1]:
        print("index yes")
  
    if row["max_pct"]>max_pct_gain:
        max_pct_gain=row["max_pct"]


#         print("===========================")
#         print("Trade index: i{} s{}, f{}, Signal direction {}".format(index, 
#                                                                       trade.index[0], 
#                                                                       trade.index[-1], 
#                                                                       trade.signal.iloc[0]))


    # Now we'll calc the tp winners

    # We need to start with an if statement per tp... elifs will break the strategy as a 
    # single candle could pass thro' tp1 to tp2 and we would not bank relevant profits at tp1!

    if row["max_pct"]>=tp1 and tp1_hit==False:

        # Assuming tp1,tp2,tp3 = 0.38,0.54,0.6 (tp1*10 =3.8%) and initial capital ==100 and p1,p2,p3 ==1,1,98
#             print(row["max_pct"], tp1)
        
        tp1_hit=True
        position = running_capital*(1+(tp1/10)) # 103.8
        profit_take = position*(p1/100) # 1.038
#             print("profit take {}".format(profit_take))
        profit_slices.append(profit_take) # Add 1.038
        # new_position = position*((100-p1)/100)
        new_position = position-profit_take # 103.8-1.038 = 102.76
#             print("new position {}".format(new_position))
        running_capital = new_position
        print(index)
        print(f"tp1hit; runcap{running_capital}, profitslice{profit_slices}")

    if row["max_pct"]>=tp2 and tp2_hit==False:
#             print(row["max_pct"], tp2)
        tp2_hit=True
        # Now we have moved from tp1 to tp2 but we can only book the difference against our position size
        price_gain = tp2-tp1 # 0.54-0.38
        position = running_capital*(1+(price_gain/10)) # 102.76*1.016 = 104.4...
        profit_take = position*(p2/100) # 104.4*(1/100) = 1.044
        profit_slices.append(profit_take) # 1.044
        # new_position = position*((1-p2)/10)
        new_position = position-profit_take # 103.362
        running_capital = new_position
        print(index)
        print(f"tp2hit; runcap{running_capital}, profitslice{profit_slices}")

    if row["max_pct"]>=tp3 and tp3_hit==False:
        tp3_hit=True
        # Now we have moved from tp2 to tp3 but we can only book the difference against our position size
        price_gain = tp3-tp2 # 0.6-0.54=0.06
        position = running_capital*(1+(price_gain/10)) # 103.362*1.006 = 103.98
        # profit_take = position*(p3/100)
        # Now we close the trade at tp3 so we take whatever profit is on the table and add to profit slice
        profit_take = position-100 # ~3.98
        # Add the profit
        profit_slices.append(profit_take)
        print(index)
        print(f"tp3hit; runcap{running_capital}, profitslice{profit_slices}")
        # new_position = position*((1-p3)/10)
        # WE NO LONGER HAVE A POSITION OPEN AT THIS POINT AS ALL RISK HAS BEEN REMOVED FROM TABLE
        #new_position = position-profit_take
        #capital = new_position


    # First we calculate the stops and losses

    # If we are on the last candle of the trade and we haven't hit any tps - we have a loser
    # The nature of the data is that the supertrend reversal / stop is hit on the last candle

    if index == trade.index[-1] and tp1_hit==False:
        ## SIMPLY IF WE ARE ON THE LAST CANDLE HAVE HIT NO TPS WE WILL STOP OUT AT THE SUPERTREND
        ## The %age drop/gain can be calculated by the entry price and the close price of last candle

        # Calculate pct loss for a buy trade stop out
        if trade.iloc[0].signal=="buy":
            factor = row['close']/entry_price
            pct_loss = (factor-1)*100
            profit_slices.append(pct_loss*10)
            print(index)
            print(f"stopped no tp {pct_loss}")

        # Calculate pct loss for a sell trade stop out
        elif trade.iloc[0].signal=="sell":
            factor = row['close']/entry_price
            pct_loss = (factor-1)*(-100)
            profit_slices.append(pct_loss*10)
            print(index)
            print(f"stopped no tp {pct_loss}")




    if index == trade.index[-1] and tp2_hit == True and tp3_hit == False:
        print("hello")

        if trade.iloc[0].signal=="buy":

            # Calculate buys closing at stop
            tp2_price = entry_price*(1+(tp2/10))
            close_price = row["close"]

            factor = close_price/tp2_price

            final_position = running_capital*factor # 103.362*0.9298 = 96.105

            pct_loss = ((final_position/initial_capital)-1)*100 # ((96.105/100)-1)*100 = -3.89

            #pct_loss = (factor-1)*100
            profit_slices.append(pct_loss)
            print(index)
            print(f"buy stopped tp2 hit {pct_loss}")


        elif trade.iloc[0].signal=="sell": 

            # Calculate sells closing at stop
            tp2_price = entry_price*(1-(tp2/10))

            close_price = row["close"]

            # The following is convoluted because we're trying to calculate a pct drop of position
            # on an actual price rise
            factor = 1-(abs(1-(close_price/tp2_price)))
            final_pos = running_capital*factor
            pct_loss = final_pos-100

            profit_slices.append(pct_loss)
            print(index)
            print(f"sell stopped tp2 hit {pct_loss}")




    elif index == trade.index[-1] and tp1_hit == True and tp2_hit==False and tp3_hit==False:
        # BUYS ... remember buys have a sell signal on there last row signalling reversal
        if trade.iloc[0].signal=="buy":

            # Calculate buys closing at stop
            tp1_price = entry_price*(1+(tp1/10))
            close_price = row["close"]

            factor = close_price/tp1_price

            final_position = running_capital*factor # 103.362*0.9298 = 96.105

            pct_loss = ((final_position/initial_capital)-1)*100 # ((96.105/100)-1)*100 = -3.89

            #pct_loss = (factor-1)*100
            profit_slices.append(pct_loss)
            print(index)
            print(f"buy stopped tp1 hit {pct_loss}")


        elif trade.iloc[0].signal=="sell":

            # Calculate sells closing at stop
            tp1_price = entry_price*(1-(tp1/10))

            close_price = row["close"]

            # The following is convoluted because we're trying to calculate a pct drop of position
            # on an actual price rise
            factor = 1-(abs(1-(close_price/tp1_price)))
            final_pos = running_capital*factor
            pct_loss = final_pos-100

            profit_slices.append(pct_loss)
            print(index)
            print(f"sell stopped tp1 hit {pct_loss}")





#         print(tp1_hit, tp2_hit, tp3_hit)
print(trade.iloc[0].time)        
print(tp1_hit, tp2_hit, tp3_hit)
print(profit_slices)
trade_profit = sum(profit_slices)
if trade_profit>100:
    outcome.append("win")
else:
    outcome.append("loss")
max_pct_trade.append(max_pct_gain)
# Append the sum of our profit list to the trade results list
results.append(trade_profit)







False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
589
tp1hit; runcap61.8, profitslice[41.2]
True False False
True False False
True False False
True False False
True False False
True False False
True False False
596
tp2hit; runcap38.19239999999999, profitslice[41.2, 25.4616]
True True False
True True False
True True False
True True False
True True False
True True False
index yes
hello
602
sell stopped tp2 hit -64.38923717334126
2022-03-14 09:19:00
True True False
[41.2, 25.4616, -64.38923

In [80]:
df[df.time.between("2022-03-14 22:00:00", "2022-03-14 22:55:00")]

,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct
1320,2022-03-14 22:00:00,38739.82,38752.28,38709.44,38738.12,NaN,38754.942988,sell,nosig,-51.11,-22.43,-0.058
1321,2022-03-14 22:01:00,38738.11,38774.25,38718.88,38770.51,38654.445116,NaN,buy,buysig,0.00,0.00,0.000
1322,2022-03-14 22:02:00,38772.44,38772.45,38709.17,38744.48,38654.445116,NaN,buy,nosig,-26.03,1.94,0.005
1323,2022-03-14 22:03:00,38744.07,38773.97,38744.02,38770.70,38658.307294,NaN,buy,nosig,0.19,3.46,0.009
1324,2022-03-14 22:04:00,38761.33,38771.98,38759.25,38771.98,38671.177064,NaN,buy,nosig,1.47,1.47,0.004
1325,2022-03-14 22:05:00,38771.97,38778.21,38757.99,38766.27,38677.039858,NaN,buy,nosig,-4.24,7.70,0.020
1326,2022-03-14 22:06:00,38766.27,38766.27,38727.60,38739.01,38677.039858,NaN,buy,nosig,-31.50,-4.24,-0.011
1327,2022-03-14 22:07:00,38738.43,38753.38,38724.67,38748.80,38677.039858,NaN,buy,nosig,-21.71,-17.13,-0.044
1328,2022-03-14 22:08:00,38741.29,38780.16,38682.08,38771.56,38677.039858,NaN,buy,nosig,1.05,9.65,0.025
1329,2022-03-14 22:09:00,38771.97,38774.70,38736.37,38755.62,38677.039858,NaN,buy,nosig,-14.89,4.19,0.011


#### SANDPIT

In [37]:
close_price=40800
tp2_price = 37840
position=103.362

factor = 1-(abs(1-(close_price/tp2_price)))
final_pos = position*factor
pct_loss = final_pos-100
pct_loss

-4.723399577167029

In [38]:
lsta = [-9, 4, -2]
sum(lsta)

-7

In [172]:
cols = ["tp1", "tp2", "tp3"]
tp_df = pd.DataFrame(tp_list, columns=cols)
tp_df

,tp1,tp2,tp3
0,True,False,False
1,True,True,True
2,True,False,False
3,False,False,False
4,False,False,False
...,...,...,...
656,True,False,False
657,False,False,False
658,True,True,False
659,False,False,False


In [173]:
trig_copy = trigs.copy()
trig_copy.reset_index(inplace=True)
trig_copy = trig_copy.join(res)
trig_copy.rename(columns={0:"results"}, inplace=True)
trig_copy = trig_copy.join(tp_df)

In [174]:
trig_copy[trig_copy["time"].between("2022-03-28 07:00:00", "2022-03-28 11:00:00")]

,index,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct,results,tp1,tp2,tp3
571,20583,2022-03-28 07:03:00,46967.18,46992.40,46965.01,46979.98,46861.556874,NaN,buy,buysig,0.0,0.0,0.0,0.628209,True,False,False
572,20646,2022-03-28 08:06:00,46954.68,46967.20,46935.01,46935.01,NaN,47039.086510,sell,sellsig,0.0,0.0,0.0,-0.937040,False,False,False
573,20667,2022-03-28 08:27:00,46964.81,46979.70,46959.66,46978.99,46886.351016,NaN,buy,buysig,0.0,0.0,0.0,1.844852,True,True,False
574,20795,2022-03-28 10:35:00,47196.62,47201.21,47084.65,47114.88,NaN,47267.528068,sell,sellsig,0.0,0.0,0.0,-2.442965,False,False,False
575,20814,2022-03-28 10:54:00,47203.13,47230.36,47185.93,47229.98,47094.278055,NaN,buy,buysig,0.0,0.0,0.0,0.593780,True,False,False


In [170]:
trig_copy[trig_copy["results"]>5]#.sort_values()

,index,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct,results
1,16,2022-03-14 00:16:00,37750.61,37846.00,37750.61,37839.84,37616.556977,NaN,buy,buysig,0.0,0.0,0.0,6.5646
7,210,2022-03-14 03:30:00,38059.70,38100.38,38059.62,38079.20,37998.075057,NaN,buy,buysig,0.0,0.0,0.0,6.5646
11,365,2022-03-14 06:05:00,38536.92,38562.33,38536.92,38562.33,38457.160864,NaN,buy,buysig,0.0,0.0,0.0,6.5646
37,1321,2022-03-14 22:01:00,38738.11,38774.25,38718.88,38770.51,38654.445116,NaN,buy,buysig,0.0,0.0,0.0,6.5646
40,1424,2022-03-14 23:44:00,39661.05,39664.71,39557.29,39560.65,NaN,39774.747183,sell,sellsig,0.0,0.0,0.0,6.5646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,21597,2022-03-28 23:57:00,47116.84,47199.99,47111.76,47190.78,46910.313161,NaN,buy,buysig,0.0,0.0,0.0,6.5646
622,22537,2022-03-29 15:37:00,47895.76,47900.00,47869.99,47878.73,NaN,48017.149796,sell,sellsig,0.0,0.0,0.0,6.5646
632,22899,2022-03-29 21:39:00,47456.45,47456.45,47430.84,47432.26,NaN,47514.605464,sell,sellsig,0.0,0.0,0.0,6.5646
636,23102,2022-03-30 01:02:00,47246.10,47249.62,47162.97,47171.06,NaN,47345.873351,sell,sellsig,0.0,0.0,0.0,6.5646


In [179]:
tp2_stop = trig_copy[(trig_copy.tp1==True) & (trig_copy.tp2==True) & (trig_copy.tp3==False)]
tp2_stop

,index,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct,results,tp1,tp2,tp3
15,560,2022-03-14 09:20:00,38997.34,39068.67,38994.06,39066.11,38906.881058,NaN,buy,buysig,0.0,0.0,0.0,1.784182,True,True,False
20,685,2022-03-14 11:25:00,39047.19,39059.01,38984.53,39004.94,NaN,39122.420846,sell,sellsig,0.0,0.0,0.0,1.670523,True,True,False
25,841,2022-03-14 14:01:00,38851.63,38910.44,38823.87,38903.12,38696.370135,NaN,buy,buysig,0.0,0.0,0.0,1.699614,True,True,False
30,1012,2022-03-14 16:52:00,38921.85,38925.21,38866.92,38881.81,NaN,39016.228542,sell,sellsig,0.0,0.0,0.0,1.537511,True,True,False
32,1095,2022-03-14 18:15:00,38713.91,38716.19,38670.00,38670.71,NaN,38797.988947,sell,sellsig,0.0,0.0,0.0,1.432748,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,22788,2022-03-29 19:48:00,47751.39,47758.07,47726.87,47730.01,NaN,47846.751546,sell,sellsig,0.0,0.0,0.0,1.690006,True,True,False
634,23040,2022-03-30 00:00:00,47448.41,47451.41,47401.00,47430.74,NaN,47526.938709,sell,sellsig,0.0,0.0,0.0,1.633706,True,True,False
641,23335,2022-03-30 04:55:00,47262.85,47266.89,47256.75,47266.89,47181.396476,NaN,buy,buysig,0.0,0.0,0.0,1.869533,True,True,False
646,23502,2022-03-30 07:42:00,47475.28,47475.28,47444.91,47447.05,NaN,47529.828748,sell,sellsig,0.0,0.0,0.0,1.516710,True,True,False


In [181]:
tp2_stop.sort_values(by="results")

,index,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct,results,tp1,tp2,tp3
130,4915,2022-03-17 09:55:00,40805.58,40808.96,40716.01,40724.53,NaN,40854.730984,sell,sellsig,0.0,0.0,0.0,1.289132,True,True,False
90,3509,2022-03-16 10:29:00,40460.45,40467.78,40430.16,40435.67,NaN,40589.375437,sell,sellsig,0.0,0.0,0.0,1.334994,True,True,False
206,7391,2022-03-19 03:11:00,41763.69,41765.31,41737.55,41744.05,NaN,41826.730999,sell,sellsig,0.0,0.0,0.0,1.364841,True,True,False
140,5268,2022-03-17 15:48:00,40909.94,40914.37,40885.34,40893.25,NaN,41004.270931,sell,sellsig,0.0,0.0,0.0,1.409387,True,True,False
102,4155,2022-03-16 21:15:00,41277.03,41279.00,41228.81,41228.81,NaN,41404.622420,sell,sellsig,0.0,0.0,0.0,1.422443,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,5599,2022-03-17 21:19:00,40776.72,40791.80,40771.43,40787.48,40717.157931,NaN,buy,buysig,0.0,0.0,0.0,1.878011,True,True,False
517,18260,2022-03-26 16:20:00,44237.73,44271.61,44228.95,44264.54,44157.340227,NaN,buy,buysig,0.0,0.0,0.0,1.887959,True,True,False
317,11252,2022-03-21 19:32:00,40974.69,41029.55,40964.85,41018.24,40896.668032,NaN,buy,buysig,0.0,0.0,0.0,1.911926,True,True,False
33,1132,2022-03-14 18:52:00,38560.81,38656.73,38558.12,38641.54,38458.662744,NaN,buy,buysig,0.0,0.0,0.0,1.942086,True,True,False


#### NARRATIVE
The results for TP2-to-stop are low... so far they seem to be half the manually backtested result

In [182]:
tp1_stop = trig_copy[(trig_copy.tp1==True) & (trig_copy.tp2==False) & (trig_copy.tp3==False)]
tp1_stop

,index,time,open,high,low,close,uptrend,downtrend,signal,trigger,close_pl,max_pl,max_pct,results,tp1,tp2,tp3
0,9,2022-03-14 00:09:00,37723.24,37725.89,37699.00,37712.00,NaN,37839.792000,sell,sellsig,0.0,0.0,0.0,0.310122,True,False,False
2,84,2022-03-14 01:24:00,38080.27,38094.26,38037.45,38059.44,NaN,38218.679324,sell,sellsig,0.0,0.0,0.0,0.519948,True,False,False
16,602,2022-03-14 10:02:00,39147.38,39155.74,39105.05,39137.00,NaN,39266.420513,sell,sellsig,0.0,0.0,0.0,0.686815,True,False,False
21,777,2022-03-14 12:57:00,38791.26,38835.97,38789.13,38826.43,38687.502350,NaN,buy,buysig,0.0,0.0,0.0,0.616165,True,False,False
26,881,2022-03-14 14:41:00,38960.72,38961.74,38915.26,38915.26,NaN,39079.289925,sell,sellsig,0.0,0.0,0.0,0.384987,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,23536,2022-03-30 08:16:00,47290.00,47351.26,47287.06,47347.22,47183.158282,NaN,buy,buysig,0.0,0.0,0.0,0.736132,True,False,False
654,23713,2022-03-30 11:13:00,47297.92,47300.78,47247.36,47257.13,NaN,47332.954211,sell,sellsig,0.0,0.0,0.0,0.462290,True,False,False
655,23717,2022-03-30 11:17:00,47187.26,47356.32,47145.37,47326.67,47094.318838,NaN,buy,buysig,0.0,0.0,0.0,0.740458,True,False,False
656,23743,2022-03-30 11:43:00,47388.35,47388.35,47345.21,47352.20,NaN,47477.820162,sell,sellsig,0.0,0.0,0.0,0.739506,True,False,False


#### POTENTIAL EXTRA PROFIT

In [190]:
tp1_add = tp1_stop.results.sum()*0.33
tp2_add = tp2_stop.results.sum()
extra = tp1_add+tp2_add
extra

153.81222490934525

In [192]:
extra - abs(sum(results))

51.99277001688188

#### NARRATIVE
For the tested parameters there is a possible ~153% extra percent up for grabs. That means our strategy would return 51% over the given time period!